In [ ]:

"""
Task 2: Data Visualisation
1. a.
"""
from time import sleep
from kafka3 import KafkaConsumer
import datetime as dt
import matplotlib.pyplot as plt

# this line is needed for the inline display of graphs in Jupyter Notebook
%matplotlib notebook
topic = 'Scenaries01'
hostip = "192.168.1.196" 



def annotate_max(x,y,ax = None):
    ymax = max(y)
    xpos = y.index(ymax)
    xmax = x[xpos]
    text = 'Max: Time={}, Value={}'.format(xmax, ymax)
    if not ax:
        ax = plt.gca()                                       # arrow position in plot
    ax.annotate(text, xy=(xmax, ymax), xytext=(xmax,ymax+5), arrowprops=dict(facecolor='red', shrink=0.02),)

def annotate_min(x,y,ax = None):
    ymin = min(y)
    xpos = y.index(ymin)
    xmin = x[xpos]
    text = 'Min: Time={}, Value={}'.format(xmin, ymin)
    if not ax:
        
        #plot in the figure
        ax = plt.gca()                                       # arrow position in plot
    ax.annotate(text, xy=(xmin, ymin), xytext=(xmin,ymin-5), arrowprops=dict(facecolor='blue', shrink=0.02),)


def connect_kafka_consumer():
    consumer = None
    try:
        consumer = KafkaConsumer(topic,
                                consumer_timeout_ms=10000,
                                bootstrap_servers=[f'{hostip}:9092'],
                                api_version=(0,10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return consumer



def init_plots():
    try:
        width = 9.5
        height = 6
        fig = plt.figure(figsize=(width, height))
        #adding subplot to figure
        #The argument 111 is a 3-digit integer where the first digit represents the 
        #number of rows, the second digit represents the number of columns, and the 
        #third digit represents the index of the subplot.
        ax = fig.add_subplot(111)
        fig.suptitle('Climate Streaming with Max, Min values')
        ax.set_xlabel('Arrival Time')
        ax.set_ylabel('Air Temperature')
        #         ax.set_ylim(0,110)
        #displace figure
        fig.show()
        #draw the canvas
        fig.canvas.draw()
        return fig, ax
    
    except Exception as ex:
        print("init_plots")
        print(str(ex))

def consume_msg(consumer, fig, ax):
#     try:
    x, y = [], []
    for msg in consumer:
        #split each stringed (key: value) pair by ',' for the value part
                #convert to string split it by ,

        # bytes -> string, 
        # msg object -> value
        # decode already -> string
        #delimited like csv
        #since encoded one set/dictionary -> 
        # data = 1 list has 1 set
        data = str(msg.value.decode('utf-8')).split(',')
        air_temp_key_value = data[2]
#         print(air_temp_tuple)
        air_temp_key_value = air_temp_key_value.split(':')
        air_temp_value = int(air_temp_key_value[1])
        #print(air_temp_value)
        y.append(int(air_temp_value))

        #arrival time
        # %X' is the format code for the locale’s appropriate time representation.
        x.append(dt.datetime.now().strftime('%X'))
        #print(x)

        # start plotting when at least 10 data points
        if len(y) > 10:
            #need to clear time to repaint to refresh the new data
            ax.clear()
            ax.plot(x,y)
            ax.set_xlabel('Time')
            ax.set_ylabel('Value')
            annotate_max(x,y,ax)
            annotate_min(x,y,ax)
            fig.canvas.draw()
            #pop the oldest value
            x.pop(0)
            y.pop(0)

    plt.close('all')
            
#     except Exception as ex:
#         print("consume_msg")
#         print(str(ex))
        
if __name__ == '__main__':
    consumer = connect_kafka_consumer()
    fig, ax = init_plots()
    consume_msg(consumer,fig, ax)